<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Catalina Contreras 
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `https://github.com/catafarro/MDS7202/blob/main/Lab9.ipynb`

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Obtener datos desde Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Optimizar a través de librerías funciones.

#Importamos librerias utiles 😸

In [1]:
%%capture
!pip install praw
!pip install line_profiler
!pip install numba

import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://styles.redditmedia.com/t5_2rer8/styles/bannerBackgroundImage_xzrb6my122181.jpg">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [38]:
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url


In [39]:
votes, post, url = praw_reddit()

In [15]:
###### Código Aquí ######
!pip install line_profiler
%load_ext line_profiler

%lprun -f   praw_reddit  praw_reddit()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Resultado : 

```
Timer unit: 1e-06 s

Total time: 12.4026 s
File: <ipython-input-2-bd47716803a4>
Function: praw_reddit at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     2         1          9.0      9.0      0.0      reddit = praw.Reddit(
     3         1          1.0      1.0      0.0          client_id="-w2hyFINxZ8T3g",
     4         1          2.0      2.0      0.0          client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
     5         1          1.0      1.0      0.0          password="ClasesMDS7202",
     6         1          1.0      1.0      0.0          user_agent="Clases",
     7         1          1.0      1.0      0.0          username="DocenciaDataScience",
     8         1        942.0    942.0      0.0          check_for_async=False,
     9                                               )
    10         1         43.0     43.0      0.0      subreddit = reddit.subreddit(nombre_subreddit)
    11                                           
    12         1          2.0      2.0      0.0      votes, post, url = {}, {}, {}
    13         1   12313462.0 12313462.0     99.3      top_submissions = list(subreddit.hot(limit=n_hot))
    14        20         36.0      1.8      0.0      for it, top_n in enumerate(range(50, len(top_submissions), 50)):
    15        19        168.0      8.8      0.0          top_n_submissions = top_submissions[:top_n]
    16        19        118.0      6.2      0.0          upvotes, downvotes, url[it], post[it] = [], [], [], []
    17                                           
    18      9519       7548.0      0.8      0.1          for submission in top_n_submissions:
    19      9500       7291.0      0.8      0.1              try:
    20      9500      10107.0      1.1      0.1                  ratio = submission.upvote_ratio
    21      9500       7729.0      0.8      0.1                  ups = int(
    22                                                               round((ratio * submission.score) / (2 * ratio - 1))
    23      9500      13829.0      1.5      0.1                      if ratio != 0.5
    24       151        175.0      1.2      0.0                      else round(submission.score / 2)
    25                                                           )
    26      9500       8423.0      0.9      0.1                  upvotes.append(ups)
    27      9500       9192.0      1.0      0.1                  downvotes.append(ups - submission.score)
    28      9500      10586.0      1.1      0.1                  post[it].append(submission.title)
    29      9500      11216.0      1.2      0.1                  url[it].append(submission.url)
    30                                                       except Exception as e:
    31                                                           continue
    32        19       1739.0     91.5      0.0          votes[it] = np.array([upvotes, downvotes]).T
    33         1          1.0      1.0      0.0      return votes, post, url

```



In [25]:
!pip install memory_profiler
%load_ext memory_profiler

%memit 

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 275.84 MiB, increment: 0.00 MiB


**Respuesta:**

Se ve que  la más tiempo demora es top_submissions = list(subreddit.hot(limit=n_hot))  

## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

**Respuestas Teóricas:**

1. La memoria cache es un tipo de memoria a la cual el procesador tiene acceso directo, es rápida de leer y pequeña. Por otro lado LRU significa Least Recently Used y es una estrategia usada para entregar espacios en memoria, en este caso de la memoría caché donde los pedazos menos usados en un intervalo de tiempo se descartan para dar más espacio a nuevos pedazos. 
2. El algoritmo tiene un costo computacional alto ya que cada elemento o pedazo que guardemos debe ser actualizado ( movido) en cada pasada y esto demora tiempo
3. Pese a que nos vemos enfrentados a este costo, aún así es más conveniente que leer desde la memoria normal por lo que el tiempo de ejecución bajara  



In [57]:
%timeit praw_reddit()

1 loop, best of 5: 11.7 s per loop


In [62]:
from functools import lru_cache

@lru_cache()
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [63]:
%%timeit
praw_reddit()

The slowest run took 69644212.67 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 184 ns per loop


**Respuesta de lru_cache sobre praw_reddit**

Cómo se comento anteriormente, el tiempo baja considerablemente al utilizar memoria cache 


## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [89]:
###### Código Aquí ######

def sin_numpy(votos):
  sigma =[]
  mu =[]
  for vote in votos : 
    a= vote[0]
    b= vote[1]
    sigma.append(1.65 *((a*b/((a+b)**2 * (a+b+1))))**0.5)
    mu.append(a / (a+b))
  return sigma , mu


def con_numpy(votos):
  votos =  1 + votos
  a= votos[:,0]
  b= votos[:,1]
  sigma = 1.65 *((a*b/((a+b)**2 * (a+b+1))))**0.5
  mu = a / (a+b)
  return sigma , mu




## 1.4 Comparación de Rendimiento [2 Puntos]

Creadas las funciones de la sección 1.3, de forma iterativa compare los diferentes batches de datos que contiene el diccionario `votes`. Para esto genere un gráfico utilizando plotly, donde se pueda observar las diferencias de tiempo que toma ejecutar las diferentes cantidades de datos. ¿es posible observar una diferencia? ¿a qué se deberá esto?.

Aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

In [91]:
###### Código Aquí ######
@jit(nopython=True)
def sin_numpy_numba(votos):
  sigma =[]
  mu =[]
  for vote in votos : 
    a= vote[0]
    b= vote[1]
    sigma.append(1.65 *((a*b/((a+b)**2 * (a+b+1))))**0.5)
    mu.append(a / (a+b))
  return sigma , mu

@jit(nopython=True)
def con_numpy_numba(votos):
  votos =  1 + votos
  a= votos[:,0]
  b= votos[:,1]
  sigma = 1.65 *((a*b/((a+b)**2 * (a+b+1))))**0.5
  mu = a / (a+b)
  return sigma , mu


In [71]:
time1=[]
time2=[]
time3=[]
time4=[]

for vote in votes : 
  sin_numpy(vote)
  con_numpy(vote)
  sin_numpy_numba(vote)
  conn_numpy_numba(vote)

SyntaxError: ignored

Ahora, grafiquen los tiempos tomados y comente los desempeños.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

In [ ]:
###### Código Aquí ######


## 1.5 Plot de Resultados [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [92]:
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
print(votos[:,0])
posterior_mean, std_err = con_numpy_numba(votos)
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post

r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()

lower bounds aproximados:
[  92   36  282  480  101  144  193  160  137   32  150   29  117   39
  249   46   22   32   61   27   16   30   45   40   72   44   13   53
   41   10   95  456   52    5  261  306  843  181  253   10   21  167
   78 1069  135  129  210   70   15  139  199   14    8   94  220   78
    5  146   74  111  369    4   78   11   33  370  216  212   68    0
   40   28  261    0  180  133  113   44   36  137 1265   49  153  124
  810  120   73   37    0   61  419   70   28  512   16   23   22   10
    5   66   25   65   62   35   10    0   29   44  265   36   24    0
    0  735  151  187  162   85  800   43  559  129  218   12  384   63
  277   53  113    9  258   57  144  392   70    0  191   90  127    0
  178  158  126   86  110   27    0   49   10  347   16   92   72   33
   25   40   38  227   72   53   15   90    0   26   31   38   15   43
   79 1016   16   61   25   16    0   23  314   12   28   20   11   13
   11   59    0    8  409  779   18    0    0   11 

**Respuesta:**

`Escriba su respuesta aquí`

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>